# Preprocessing v2

### Steps:

1. **Load Datasets:**
   - Load 'charts_processed.csv'.

2. **Align Datasets:**
   - Rename 'id' in audio features to 'track_id'.
   - Keep only top 200 chart entries.

3. **Add Columns:**
   - Extract 'track_id' from chart URLs.

4. **Remove Columns:**
   - Drop unnecessary columns ('url', 'chart', 'trend').

5. **Calculate Streams Percentage:**
   - Create 'streams_percentage' in charts.
   - Calculate percentage for each row.

6. **Validate Data:**
   - Check if sampled date-region 'streams_percentage' sums close to 1.0.

7. **Save CSV:**
   - Save preprocessed data as 'charts_processed_v2.csv'.

Note: This version ensures dataset alignment, calculates streams percentage and validates data integrity. The result is saved for future use.


In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
KAGGLE = True

In [3]:
if KAGGLE:
    CHARTS_PATH = '/kaggle/input/regionalrhythms/charts_processed.csv'
    PATH_TO_SAVE = '/kaggle/working/'
else:
    CHARTS_PATH = "../../data/charts_processed.csv"
    PATH_TO_SAVE = "../../data/"

In [4]:
def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b: break
        yield b

num_lines = 0
with open(CHARTS_PATH, "r",encoding="utf-8",errors='ignore') as f:
    num_lines = sum(bl.count("\n") for bl in blocks(f))
    
# Load the datasets into dataframes
with tqdm(total=int(num_lines)) as bar:
    charts_df = pd.read_csv(CHARTS_PATH, parse_dates=['date'], date_format='%Y-%m-%d', skiprows=lambda x: bar.update(1) and False)
charts_df.head()

Reading number of lines


  0%|          | 0/26173376 [00:00<?, ?it/s]

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [5]:
# For now only restrict the dataset to top200 charts for stream/ranking analysis.
charts_df = charts_df[charts_df["chart"] == "top200"]
charts_df["track_id"] = charts_df["url"].apply(lambda x: x.split("/")[-1])

# drop the url, chart and trend columns
charts_df.drop(columns=["url", "chart", "trend"], inplace=True)

# Create a new column for streams_percentage
charts_df['streams_percentage'] = 0.0

# Create a dictionary to store total streams for each region-date combination
total_streams_dict = {}

# Populate the dictionary
for (region, date), group in tqdm(charts_df.groupby(['region', 'date'])):
    total_streams_dict[(region, date)] = group['streams'].sum()

charts_df['streams_percentage'] = charts_df.progress_apply(lambda row: row['streams'] / total_streams_dict.get((row['region'], row['date']), 0), axis=1)

charts_df.head()

  0%|          | 0/109991 [00:00<?, ?it/s]

  0%|          | 0/20321858 [00:00<?, ?it/s]

,title,rank,date,artist,region,streams,track_id,streams_percentage
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,Argentina,253019.0,6mICuAdrwEjh6Y6lroV2Kg,0.032073
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,Argentina,223988.0,7DM4BPaS7uofFul3ywMe46,0.028393
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,Argentina,210943.0,3AEZUABDXNtecAOSC1qTfo,0.026739
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,173865.0,6rQSrBHf7HlZjtcMZ4S4bO,0.022039
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,Argentina,153956.0,58IL315gMSTD37DOZPJ2hf,0.019516


In [6]:
# first build 200 date-region combinations
date_region_combinations = list(charts_df.groupby(["date", "region"]).groups.keys())

# sample 200 combinations
indices = np.random.choice(len(date_region_combinations), 200)
sampled_date_region_combinations = [date_region_combinations[i] for i in indices]

# check if the streams_percentage adds up to 1, dont worry about the rounding errors print if the sum is not 1
for date, region in tqdm(sampled_date_region_combinations):
    df = charts_df[(charts_df["date"] == date) & (charts_df["region"] == region)]
    if not np.isclose(df["streams_percentage"].sum(), 1.0):
        print("Sum is not 1.0")

  0%|          | 0/200 [00:00<?, ?it/s]

In [7]:
# save the csv file
charts_df.to_csv(PATH_TO_SAVE + "charts_processed_v2.csv", index=False)